In [ ]:
import os
# Gets the absolute path of the current notebook (works reliably if run inside the notebook itself)
notebook_path = os.path.abspath("")
# Get the parent directory (one level up)
parent_dir = os.path.abspath(os.path.join(notebook_path, ".."))
# Change working directory to the parent directory
os.chdir(parent_dir)

# import example model
import pyomo.environ as pyo
from electric_emission_cost import costs
from examples.pyomo_battery_model import BatteryPyomo
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Pyomo Example

### Specify dynamics model

Begin by specifying the dynamics model as a pyomo Concrete model. More information about the battery model is contained in `examples/pyomo_battery_model.py`

In [ ]:
# Define the parameters for the battery model
battery_params = {
    "start_date": "2022-07-01 00:00:00",
    "end_date": "2022-08-01 00:00:00",
    "timestep": 0.25,   # 15 minutes defined in hours
    "rte": 0.86,
    "energycapacity": 100,
    "powercapacity": 50,
    "soc_min": 0.05,
    "soc_max": 0.95,
    "soc_init": 0.5,
}

# Create a sample baseload profile based on a sine wave
baseload = np.sin(np.linspace(0, 4 * np.pi, 96))*100 + 1000 + np.random.normal(0, 10, 96)
# baseload = np.random.normal(1000, 20, size=96)

# Create an instance of the BatteryOpt class
battery = BatteryPyomo(battery_params, baseload, baseload_repeat=True)

# create the model on the instance battery
battery.create_model()

In [ ]:
baseload

### Retrieve and add tariffs

Retrieve example tariff sheet contained in electric_emission_cost and read it as a dataframe


In [ ]:
path_to_tariffsheet = "electric_emission_cost/data/tariff.csv"
tariff_df = pd.read_csv(path_to_tariffsheet, sep=",")
tariff_df

Using the start and end time within the battery model and the rate dataframe, create a dictionary of charges. 

*Note: the start and end time must be datetime objects.*

In [ ]:
# get the charge dictionary
charge_dict = costs.get_charge_dict(
    battery.start_dt, battery.end_dt, tariff_df, resolution="15m"
)
charge_dict

In [ ]:
# this can also be done in a dataframe format that drops all the unnecessary columns
charge_df = costs.get_charge_df(battery.start_dt, battery.end_dt, tariff_df, )
charge_df.head()

### Add cost rules to model

Set up `consumption_data_dict` to track the variable in the model that represents the total power consumption.

In [ ]:
# this example tariff only has electric utility types so we do not pass the gas key
consumption_data_dict = {"electric": battery.model.net_facility_load}

Using the baseline consumption and the rules of the rate structure, the package will find the charge categories that are relevant for the simulation time frame. It will also automatically build constraints to calculate electricity cost.

In [ ]:
# monthly total consumption - divided by 4 because of 15-min resolution
consumption_estimate = sum(baseload) / 4
battery.model.electricity_cost, battery.model = costs.calculate_cost(
    charge_dict,
    consumption_data_dict,
    resolution="15m",
    prev_demand_dict=None,
    prev_consumption_dict=None,
    consumption_estimate=consumption_estimate,
    desired_utility="electric",
    desired_charge_type=None,
    model=battery.model,
)

### Create the objective on the model and solve

In [ ]:
# create an attribute objective based on the electricity cost
battery.model.objective = pyo.Objective(
    expr=battery.model.electricity_cost,
    sense=pyo.minimize,
)

In [ ]:
# use the glpk solver to solve the model - (any pyomo-supported LP solver will work here)
solver = pyo.SolverFactory("glpk")
results = solver.solve(battery.model, tee=False) # turn tee=True to see solver output

### Visualize Output

Visualize charge arrays

In [ ]:
# create a subset of the charge_df for energy and demand charges
energy_charge_df = charge_df.filter(like="energy")
demand_charge_df = charge_df.filter(like="demand")

# sum across all energy charges
total_energy_charge = energy_charge_df.sum(axis=1)

In [ ]:
# plot 
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
# plot the energy charges
ax[0].plot(charge_df["DateTime"], total_energy_charge)
ax[0].set(xlabel="DateTime", ylabel="Energy Charge ($/kWh)", xlim=(battery.start_dt, battery.end_dt))

# plot the demand charges
ax[1].plot(charge_df["DateTime"], demand_charge_df)
ax[1].set(xlabel="DateTime", ylabel="Demand Charge ($/kWh)", xlim=(battery.start_dt, battery.end_dt), ylim=[0,None])

fig.align_ylabels()
fig.tight_layout()
fig.suptitle("Electricity Charges",y=1.02, fontsize=16)
plt.show()


Visualize model outputs

In [ ]:
net_load = np.array([battery.model.net_facility_load[t].value for t in battery.model.t])
baseload = np.array([battery.model.baseload[t] for t in battery.model.t])

fig, ax = plt.subplots()
ax.step(charge_df["DateTime"], net_load, color="C0", lw=2, label="Net Load")
ax.step(charge_df["DateTime"], baseload, color="k", lw=1, ls='--', label="Baseload")
ax.set(xlabel="DateTime", ylabel="Power (kW)", xlim=(battery.start_dt, battery.end_dt))
plt.xticks(rotation=45)
fig.tight_layout()
plt.legend()

In [ ]:
# plot the battery charge
battery_charge = np.array([battery.model.soc[t].value for t in battery.model.t])
fig, ax = plt.subplots()
ax.step(charge_df["DateTime"], battery_charge, color="C1", lw=2, label="Battery SOC")
ax.set(xlabel="Time", ylabel="Battery SOC [%]", ylim=[0,1], xlim=(battery.start_dt, battery.end_dt))
plt.xticks(rotation=45)
fig.tight_layout()

We can also just pass the numpy arrays to calculate costs to get total cost without a model or any optimization. 

In [ ]:
# pass numpy arrays for baseload and net_load (with battery) to the calculate_cost function
baseline_electricity_cost, _ = costs.calculate_cost(charge_dict = charge_dict,
                                consumption_data_dict = {"electric": baseload},
                                resolution="15m",
                                prev_demand_dict=None,
                                prev_consumption_dict=None,
                                consumption_estimate=0,
                                desired_utility="electric",
                                desired_charge_type=None,
                                model=None)

optimized_electricity_cost, _ = costs.calculate_cost(charge_dict = charge_dict,
                                consumption_data_dict = {"electric": net_load},
                                resolution="15m",
                                prev_demand_dict=None,
                                prev_consumption_dict=None,
                                consumption_estimate=0,
                                desired_utility="electric",
                                desired_charge_type=None,
                                model=None)

print(f"Baseline Electricity Cost: ${baseline_electricity_cost:.2f}")
print(f"Optimized Electricity Cost: ${optimized_electricity_cost:.2f}")


In [ ]:
# create a subset of the charge_df for energy and demand charges
charge_df = costs.get_charge_df(datetime(2023, 4, 9), datetime(2023, 4, 11), tariff_df)
charge_df.head()

energy_charge_df = charge_df.filter(like="energy")
demand_charge_df = charge_df.filter(like="demand")

# sum across all energy charges
total_energy_charge = energy_charge_df.sum(axis=1)

fig, ax = plt.subplots(2, 1, figsize=(10, 8))
# plot the energy charges
ax[0].plot(charge_df["DateTime"], total_energy_charge)
ax[0].set(xlabel="DateTime", ylabel="Energy Charge ($/kWh)", xlim=(datetime(2023, 4, 9), datetime(2023, 4, 11)))

# plot the demand charges
ax[1].plot(charge_df["DateTime"], demand_charge_df)
ax[1].set(xlabel="DateTime", ylabel="Demand Charge ($/kWh)", xlim=(datetime(2023, 4, 9), datetime(2023, 4, 11)), ylim=[0,None])

fig.align_ylabels()
fig.tight_layout()
fig.suptitle("Electricity Charges",y=1.02, fontsize=16)
plt.show()